In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
#models to explore, rbf neural network,  m-estimator based rbf, backprop nn

In [ ]:
'''need to figure out when to start my analysis from, ideas could either be 1. at the start of volcker's tenure 2. after volcker got 
inflation under control in late 1982-1983 3. during greenbaum's tenure '''

In [ ]:
#need to select features to use 

In [ ]:
#need to find some way to account for the zero lower bound

In [ ]:
#need to check for stationary and autocorrelation

In [ ]:
#features to check out, GDP, Inflation, yield curve, unemployment rate, M1, velocity of M1, industrial production, PCE (might be too 
#collinear with other indicators to be useful), retail sales, oil prices, Euro to dollar exchange rate, yen to dollar exchange rate, 
# m2, velocity of m2, real disposable personal income, imports, exports, recession status, nasdaq (maybe), price of gold, 